# Willkommen bei OutboundAI - dem ersten AI-basierten Sales Mitarbeiter deiner Firma
## Ein erster Test mit GPT und MS Azure Speech Service in Aktion
1. Die 'system_message' beinhaltet die Instruktionen an das LLM (GPT).
2. Gib deine Nachricht an das LLM in der Konsole ein. Z.B. "Hi, hier ist Marcel Kintscher von Johnson & Johnson.
3. Das Modell generiert eine Antwort auf Basis der Instruktionen in der 'system_message'.
4. Die Antwort wird mithilfe von MS Azure Speech Service in einen Audio-Output umgewandelt.
### Informationen zu den Klassen:
#### LLM (llm.py)
Diese Klasse initiiert das LLM.
Folgende Parameter müssen übergeben werden:
- model (z.B. "gpt-3.5-turbo")
- temperature (z.B. "0")
- max_tokens (z.B. "500")
- delimiter (z.B. "####")
- system_message
- user_message (vom Nutzer eingegeben)
mehr Informationen zu den GPT Parametern hier: https://platform.openai.com/docs/guides/gpt
#### TTS (speech_synthesis.py)
Diese Klasse initiiert den TTS Service.
Folgende Parameter müssen übergeben werden:
- voice_config (z.B. "de-DE-KasperNeural")
- bot_response (vom LLM übergeben)
mehr Informationen zu den TTS Parametern hier: https://learn.microsoft.com/en-us/azure/ai-services/speech-service/index-text-to-speech
### Umgebungsvariablen
- openai_secret_key
- azure_speech_key
- azure_region_key

## Klassifizierung der Anfrage

In [10]:
import os
import json
import src.llm as llm
import src.speech_synthesis as tts

# llm parameters
model = "gpt-3.5-turbo"
temperature = 0
max_tokens = 500
delimiter = "####"

# query classification
classify_system_message = f"""
Du erhältst Antworten und Fragen von einem \
potenziellen Kunden.
Die Antworten und Fragen werden durch \
folgende Zeichen abgetrennt sein: {delimiter}.
Du klassifizierst jede Antwort oder Frage in \
eine der folgenden primären und sekundären \
Kategorien. 
Bitte gib deinen Output im JSON \
Format aus mit den Schlüsseln: primary \
und secondary.

Primäre Kategorien: Einwand oder \
Generelle Antwort/Frage

Einwand sekundäre Kategorien:
Kunde hat keine Zeit
Kunde hat keine Ressourcen
Kunde hat kein Interesse

Generelle Antwort/Frage sekundäre Kategorien:
Begrüssung
Produktinformationen
Unternehmensinformationen
Preismodell
Kundenreferenzen
"""

print("Du bist der potenzielle Kunde. Gib deine Nachricht an \
      den Bot ein.")
user_message = input()

classify_llm = llm.Llm(delimiter, classify_system_message, user_message,
                  model, temperature, max_tokens)
classify_messages = classify_llm.create_prompt()
classify_json = json.loads(classify_llm.get_completion_from_messages())


if classify_json['primary']=="Generelle Antwort/Frage":
    if classify_json['secondary']=="Begrüssung":
      specify_system_message = f"""Dein Name ist Kasper.
      Du bist ein Sales Mitarbeiter in einem Software Start-up.
      
      Du bist gerade in einem Outbound Sales Telefonat mit einem \
      potenziellen Kunden. Dein Ziel ist es, diesem potenziellen \
      Kunden unser Produkt schmackhaft zu machen und ihn oder sie \
      davon zu überzeugen, einen Termin mit einem der \
      Produktexperten abzumachen.
      
      Bitte fasse deine Antworten möglichst kurz und \
      prägnant. Erfinde keine Informationen zu unserem Produkt \
      oder unserem Unternehmen. Du antwortest stets freundlich \
      und deine Antworten sollten sich so natürlich und fliessend \
      wie möglich anhören. Wenn der potenzielle Kunde eine \
      Detailfrage hat, die du nicht beantworten kannst, verweise \
      ihn oder sie stets freundlich auf die Option einen Termin \
      mit einem der Produktexperten abzumachen.

      Du bist jetzt mit dem potenziellen Kunden verbunden. \
      Wenn der potenzielle Kunde dich begrüsst, stelle dich vor \
      und erkläre warum du anrufst.
      
      Hier sind die Details zu dem Software Unternehmen und dem \
      Produkt: 
      Name: Calibo
      Industrie: Software Entwicklung
      Software Typ: Platform-as-a-Service
      Elevator Pitch: Calibo ist die erste selbstbedienbare \
      Plattform, die den Entwicklungsprozess von Digital- und \
      Datenlösungen beschleunigt, indem sie den vollständigen \
      Entwicklungszyklus über das gesamte digitale Ökosystem \
      hinweg integriert.
      Produkt Features: Product Release Orchestration Engine, Integrated \
      DevSecOps Platform, Data Intelligence Studio \
      Kundenvorteile: 50% schnellere Einführung von Digital- und \
      Datenlösungen, 40% höhere Produktivität von Entwicklerteams
      Referenzkunden: NatureSweet (ein Hersteller von frischen Lebensmitteln),
      Novartis (ein grosser Pharmakonzern), Dell Technologies \
      (ein grosses Computer-Technologie Unternehmen)
      """
      specify_llm = llm.Llm(delimiter, specify_system_message, user_message,
                  model, temperature, max_tokens)
      specify_messages = specify_llm.create_prompt()
      specify_bot_response = specify_llm.get_completion_from_messages()

Du bist der potenzielle Kunde. Gib deine Nachricht an       den Bot ein.


## Ausgabe der LLM-Antwort als Voice

In [11]:
# tts parameters
voice_config = 'de-DE-KasperNeural'

new_tts = tts.Tts(voice_config,specify_bot_response)
new_tts.speak_text()

Speech synthesized for text [Hallo Marcel, mein Name ist Kasper und ich rufe von Calibo an. Wir sind ein Softwareunternehmen, das sich auf die Entwicklung von Plattform-as-a-Service-Lösungen spezialisiert hat. Wie kann ich Ihnen helfen?]
